# Modelo Preditivo: SARIMAX (Novos dados)

O modelo SARIMAX (Seasonal Autoregressive Integrated Moving Average with eXogenous factors) é uma extensão do modelo ARIMA que inclui componentes sazonais e a capacidade de incluir variáveis exógenas. Ele é utilizado para previsão de séries temporais, permitindo capturar padrões sazonais e tendências.



## Instruções para uso local ou remoto (Google Colab ou VScode)

Aqui deixaremos brevemente um passo a passo para que você usuário seja capaz de executar o código localmente ou remotamente pelo seu google drive, podendo escolher a forma mais viável para seu uso e conhecimento.

**Google Colab**

1. Faça o upload do seu arquivo .ipynb para o Google 1.Drive.
2. Abra o Google Colab em seu navegador.
3. Clique em "Arquivo" no menu superior e selecione "Abrir notebook".
4. Na guia "Upload", clique em "Procurar" e selecione o arquivo .ipynb que você enviou para o Google Drive.
5. Após selecionar o arquivo, clique em "Abrir".
6. Aguarde o carregamento do notebook no Google Colab.
7. Agora que você carregou o notebook no Google Colab, você pode fazer as alterações necessárias nos arquivos e caminhos para se adequar ao seu ambiente específico.

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook. Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

* Descomente as linhas que começam com # from google.colab import drive;
* Comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("/content/drive/MyDrive/NomeDaPastaDosArquivos/arquivo.csv)

Salve as alterações no notebook. Agora você pode executar as células do notebook no Google Colab e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você não precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente, dado que ao executar pela ferramenta do google essas dependências estarão aplicadas por padrão

**VScode**

1. Coloque os arquivos CSV dentro da pasta "data" desse notebook

No notebook, a célula seguinte à essa contém as leituras dos arquivos CSV com o caminho do drive do criador desse notebook. Comente as linhas que fazem referência aos arquivos locais e descomente as linhas que fazem referência ao Google Drive. Por exemplo:

* Descomente as linhas que começam com # from google.colab import drive;
* Comente as linhas que começam com tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv").

Certifique-se de que os arquivos CSV estejam localizados no diretório correto em seu ambiente virtual. Por exemplo, se você tiver uma pasta chamada "data" no mesmo diretório do notebook, coloque os arquivos CSV nessa pasta e ajuste seus nomes. Possivelmente os arquivos vão seguir o seguinte padrão, mesmo no seu drive:

tabela_Meta = pd.read_csv("./data/Cópia de BASE INTELI_META_OCUP-limpo.csv")

Salve as alterações no notebook. Agora você pode executar as células do notebook no VScode e as alterações nos arquivos e caminhos serão aplicadas ao seu ambiente do colab. Lembre-se de que você precisará ter as bibliotecas necessárias instaladas em seu ambiente local para executar o código corretamente:

Para fazer a instalação, basta abrir o terminal integrado e inserir o seguinte:

In [57]:
# pip install pandas numpy matplotlib scikit-learn pmdarima

## Execução do Modelo

Aqui, importamos as bibliotecas necessárias. O SARIMAX é importado da biblioteca statsmodels, que é usada para modelagem estatística.

In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm
from copy import deepcopy
import pmdarima as pm
from pmdarima import auto_arima
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

Abaixo, usei os arquivos na pasta "data" com os nomes descritos, mas lembre-se de mudar conforme necessecidade;


In [ ]:
# # pegando arquivos csv do drive
# from google.colab import drive
# drive.mount('/content/drive')

# # Lendo os arquivos CSV
# tabela_Meta = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/BASE INTELI_META-limpo.csv")
# tabela_Agosto = pd.read_csv("/content/drive/MyDrive/Primeiro Ano/Módulo 3 - Modelo Preditivo Gazeta/Base de Dados Limpas/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")

# para realizar o processo localmente, descomente as linhas abaixo e comente as linhas acima.
tabela_Meta = pd.read_csv("../data/dados_tratados/BASE INTELI_META-limpo.csv")
tabela_Agosto = pd.read_csv("../data/dados_tratados/tratada_BaseDados_ProjetoINTELI_RG_01_AGOSTO_2024.csv")
df_dig = pd.read_csv('../data/dados_tratados/BancoReceitas_Out2024_limpa_DIG.csv')
df_rd = pd.read_csv('../data/dados_tratados/BancoReceitas_Out2024_limpa_RD.csv')
df_tv = pd.read_csv('../data/dados_tratados/BancoReceitas_Out2024_limpa_TV.csv')

df_dig = df_dig.drop('index', axis=1)
df_rd = df_rd.drop('index', axis=1)
df_tv = df_tv.drop('index', axis=1)

# Combina os três dataframes em um só
df_combined = pd.concat([df_dig, df_rd, df_tv], ignore_index=True)
print(df_dig)

O formato de exibição das tabelas é definida e padronizada, transformando as colunas de "Mês" e "Ano" em strings para sua concatenação em uma segunda coluna "Sazonalidade", que segue o formato datetime da biblioteca Pandas e as demais colunas em numéricas.

In [ ]:
import pandas as pd

# Configura o formato de exibição para floats
pd.set_option('display.float_format', '{:.0f}'.format)

# Função para limpar e converter valores
def clean_value(value):
    try:
        return str(int(float(value)))  # Remove decimais e converte para string
    except ValueError:
        return None  # Retorna None se não puder converter

# Combina os três dataframes em um só
df_combined = pd.concat([df_dig, df_rd, df_tv], ignore_index=True)
df_combined = df_combined.drop('Unnamed: 0', axis=1)

# Para df_dig
df_combined['Ano'] = df_combined['Ano'].apply(clean_value)
df_combined['Mês'] = df_combined['Mês'].apply(clean_value)
df_combined = df_combined.dropna(subset=['Ano', 'Mês'])  # Remove linhas com valores inválidos
df_combined['Sazonalidade'] = pd.to_datetime(df_combined['Ano'] + '-' + df_combined['Mês'], format='%Y-%m', errors='coerce')

print(df_combined.head())

Nesse bloco é realizada a limpeza da tabela, utilizando apenas as colunas "Sazonalidade", "Ano", "Mês" e "Vl Liquido Final", referentes à série temporal dos valores líquidos mensais. Os meses são mantidos para estabelecer um controle da sazonalidade momentaneamente na análise do código.
<br>
<br>
Também é agrupado os valores de "Vl Liquido Final" que possuem a mesma sazonalidade (mês-ano semelhantes), cumprindo com uma das requisições do SARIMAX de não duplicar séries temporais. Dessa forma, é calculado o valor líquido total de todos os veículos e segmentos em cada mês.


In [ ]:
# Remove possíveis caracteres indesejados e converte a coluna para numérico
df_combined['Vl Liquido Final'] = pd.to_numeric(df_combined['Vl Liquido Final'].replace('[^0-9.]', '', regex=True), errors='coerce')

# Reagrupa os dados e realiza a soma correta
df_combined = df_combined.groupby(['Sazonalidade', 'Ano', 'Mês']).agg({
    'Vl Liquido Final': 'sum'
}).reset_index()

# Mostra as primeiras linhas do dataframe resultante
print(df_combined.head(20))

O gráfico abaixo ilustra a série temporal seguida pelo valor líquido final das receitas, o que possibilita a visualização dos padrões e possíveis _outliers_ que possam interferir no modelo.
<br>
<br>
Apesar de serem observados valores com varianças notáveis, não é favorável a remoção desses dados, uma vez que o _dataset_ é naturalmente pequeno e esses dados discrepantes são necessários para entender tendências ao longo do ano.

In [ ]:
serie_temporal = df_combined['Vl Liquido Final']
serie_temporal.plot()
plt.show()

É criado um _dataframe_ chamado resultado que utiliza apenas os dados de resultado e sazonalidade, _features_ que serão utilizadas na criação do modelo.

In [63]:
resultado = df_combined.groupby('Sazonalidade')['Vl Liquido Final'].sum().reset_index()

A partir do _dataframe_ resultado é calculado o tamanho dos _datasets_ de treinamento e teste, que serão utilizados para entender futuramente a acurácia do modelo. O valor 78% foi escolhido para garantir 12 dados a serem utilizados como teste do modelo, ou seja, um período de um ano.

In [64]:
tamanho_treino = int(len(resultado) * 0.78)
treino = resultado.iloc[:tamanho_treino]
teste = resultado.iloc[tamanho_treino:]

O *auto_arima* é uma abordagem automatizada para selecionar os melhores parâmetros de um modelo ARIMA. Ele considera os seguintes parâmetros na sua avaliação:

* Termos autorregressivos (p);
* Número de diferenciações (d);
* Número de termos de média móvel (q);
* Número de termos autorregressivos sazonais (P);
* Número de diferenciações sazonais (D);
* Número de termos de média móvel sazonais (Q);
* Periodicidade sazonal;

O *auto_arima* calculou que o melhor modelo ARIMA para as _features_ selecionadas é o ARIMA(2,0,2)(0,1,1)[12], que considera respectivamente cada um dos parâmetros acima.

In [ ]:
auto_model = auto_arima(treino['Vl Liquido Final'], seasonal=True, m=12, trace=True)

É então utilizado o modelo ARIMA(2,0,2)(0,1,1)[12] para calcular a previsão dos valores líquidos pelos próximos 12 meses.

In [66]:
forecast = auto_model.predict(n_periods=12)

O modelo passa a funcionar, armazenando os dados obtidos com as previsões do ARIMA na coluna 'previsao' durante um período de 12 meses, um ano, que serão comparados posteriormente aos dados de teste.

In [ ]:
forecast_df = pd.DataFrame(
      {'Sazonalidade': pd.date_range(start=treino['Sazonalidade'].iloc[-1], periods=13, freq='M')[1:],
       'previsao':     forecast}
)

É criado o gráfico "Previsão da receita líquida da Rede Gazeta", que considera os dados de treinamento, coloridos em laranja, os dados de teste, coloridos em azul, e as previsões obtidas pelo modelo ARIMA, coloridos em verde.
<br>
<br>
Essa representação visual facilita o entendimento das estimativas do modelo, comparando os valores dados e os valores criados para os últimos 12 meses.

In [ ]:
# Garantir que estamos apenas pegando os primeiros 12 valores de 'Sazonalidade' para corresponder com a previsão
sazonalidade_teste = teste['Sazonalidade'][:len(forecast_df)]

plt.figure(figsize=(10,6))
plt.plot(treino['Sazonalidade'], treino['Vl Liquido Final'], label='Dados de treinamento', color='orange')
plt.plot(teste['Sazonalidade'], teste['Vl Liquido Final'], label='Dados de teste', color='blue')
plt.plot(sazonalidade_teste, forecast_df['previsao'], label='Previsão', color='green')

plt.title('Previsão da receita líquida da Rede Gazeta')
plt.legend()
plt.show()

Como forma de testar a confiabilidade do modelo, são utilizados parâmetros de acurácia, dessa forma, observa-se:
<br>
<br>
R² (Coeficiente de Determinação): Métrica que calcula o quanto das variações dos dados é captada pelo modelo. Nesse dado, quanto maior o valor obtido, melhor o entendimento da variabilidade.
<br>
<br>
MSE (Erro Quadrático Médio): Calcula a média de diferença entre o valor sintético e o real, por ser elevada ao quadrado, essa métrica tende a penalizar _outliers_, revelando um valor naturalmente alto. Quanto menor o grau de erro, melhores são as previsões do modelo.
<br>
<br>
MAE (Erro Médio Absoluto): Calcula a média absoluta da diferença entre os valores previstos pelo modelo e os valores dados. Essa métrica não é influenciada pela presença de _outliers_ e revela numéricamente a diferença média dos dados. Quanto menor seu grau, mais confiáveis são as previsões.
<br>
<br>
MAPE (Erro Percentual Absoluto Médio): Essa métrica calcula percentualmente a diferença entre os valores previstos e os valores reais. A variança percentual pode ser tanto positiva quanto negativa. Quanto menor seu grau, melhores são as previsões.

In [ ]:
forecast = auto_model.predict(n_periods=len(teste))

forecast_df = pd.DataFrame(
      {'Sazonalidade': teste['Sazonalidade'],
       'previsao': forecast})

teste = teste.merge(forecast_df, on='Sazonalidade')
y_true = teste['Vl Liquido Final']
y_pred = teste['previsao']

r2 = r2_score(y_true, y_pred)
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
mape = (abs((y_true - y_pred) / y_true).mean()) * 100

print(f'R²: {r2:.2f}')
print(f'MSE: {mse:.2f}')
print(f'MAE: {mae:.2f}')
print(f'MAPE: {mape:.2f}%')

Os valores obtidos das métricas são relevantes para a utilização futura do modelo, uma vez que indicam o que deve ser levado em consideração na hora de interpretar seus resultados.
<br>
<br>
Coeficiente de Determinação: Obtendo um valor de 0,65 é possível afirmar que 65% das variações dos dados são capturadas pelo modelo. Esse resultado desempenha um nível razoável, que pode ser melhorado com a utilização de mais dados para treinamento.
<br>
<br>
Erro Médio Quadrático: Com um valor de 1.231.392.478.829,90 nota-se que existe uma diferença considerável dos dados previstos e dos dados reais. Tirando sua raíz quadrada, obtêm-se que, ao analisar os dados obtidos pelo modelo, deve ser considerada uma margem de erro de 1.109.681 para mais ou para menos.
<br>
<br>
Erro Médio Absoluto: O valor de 836.222,45 indica que o modelo erra os números reais e preditos em média por 836 mil reais aproximadamente. Apesar dos dados serem razoáveis para a escala apresentada pela Rede Gazeta, é observada uma necessidade de melhoria nessa métrica.
<br>
<br>
Erro Percentual Absoluto Médio: Com 10,13% de erro, o modelo está errando suas previsões em cerca de 10% em relação aos valores reais, isso significa que os valores previstos podem ser maiores ou menores que os reais. O MAPE é considerado bom nesse nível, no entanto, pode ser melhorado com a utilização de mais hiperparâmetros.

Por fim, é feito o gráfico de resíduos do modelo, ou seja, a diferença prática entre os valores reais e previstos (valores teste - valores de previsão). Os dados obtidos indicam uma grande variação entre os números, o que pode indicar uma tendência sazonal na série temporal. Além disso, é observada a presença de _outliers_, como os valores -1.679.975 e 10.326.114, que podem influenciar na confiabilidade do modelo.

In [ ]:
model = SARIMAX(treino['Vl Liquido Final'], order=(2, 0, 2), seasonal_order=(0, 1, 1, 12))
results = model.fit()

fitted_values = results.fittedvalues

residuos = treino['Vl Liquido Final'] - fitted_values

plt.figure(figsize=(10,6))
plt.plot(residuos)
plt.title('Resíduos do modelo SARIMAX')
plt.axhline(0, linestyle='--', color='red')
plt.show()

print(residuos.describe())

Em conclusão, nota-se que o SARIMAX não é a melhor escolha para um _dataset_ pequeno, uma vez que as tendências sazonais podem ser melhor explicadas com espaços de tempo maiores e mais homogêneos. Apesar das métricas razoáveis, é necessária a análise de outros modelos preditivos, priorizando a qualidade das previsões e sua confiabilidade.